### This notebook is mostly continued in Additional Analysis

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

"We analyzed data from 7,321 of the 11,878 participants. This subset resulted from the
following exclusion, which attempted to eliminate incomplete data or data that were already
flagged as problematic by the ABCD study organizers. First, we attempted to download 8,811
participants from the “FastTrack Recommended Active Series” from the NIMH Data Archive,
which has gone through some quality assurance for the associated imaging files by the ABCD
study organizers. Of these, 8,776 files were successfully downloaded, but a subset did not
include stop-signal data, leaving ​ 7,906 subjects. Of these, only 7,347 included ​ summary scores
from the Stop Signal Task in the ABCD Data Release 2.0. Finally, 26 subjects were removed who did not have two complete runs with 180 trials each, leaving us with a total of 7,321
complete datasets."

The first issue to look into is the question of which subset of subjects the authors used and if the authors employed the provided behavioral perforance flags.

This script is provided seperatly as I did not want to modify the chunk of the authors original code too much, so this script covers the computationally intensive part.

In [2]:
# All I could download
v1 = pd.read_csv('merged_data/all_concat.csv', low_memory=False)
len(v1.src_subject_id.unique())

8794

In [3]:
# All .txt w/o any errors
v2 = pd.read_csv('merged_data/SST_concat.csv', low_memory=False)
len(v2.src_subject_id.unique())

8123

In [4]:
# The release 2.0 summary csv, set summary_2_subjects as all subjects
summary_2_df = pd.read_csv('/home/sage/work/ABCD2p0NDA/abcd_sst02.txt', sep='\t', skiprows=[1])
summary_2_subjects = pd.unique(summary_2_df.subjectkey)
len(summary_2_subjects)

9598

In [5]:
# Alternately, set a smaller subset as those that pass the performance flag
behav_flag_subjects = pd.unique(summary_2_df[summary_2_df['tfmri_sst_beh_performflag'] == 1].subjectkey)
len(behav_flag_subjects)

8484

In [6]:
v3 = v1[v1['src_subject_id'].isin(summary_2_subjects)]
len(v3.src_subject_id.unique())

8180

In [7]:
v4 = v2[v2['src_subject_id'].isin(summary_2_subjects)]
len(v4.src_subject_id.unique())

7550

In [8]:
v5 = v1[v1['src_subject_id'].isin(behav_flag_subjects)]
len(v5.src_subject_id.unique())

7228

In [9]:
v6 = v2[v2['src_subject_id'].isin(behav_flag_subjects)]
len(v6.src_subject_id.unique())

6700

In [10]:
def get_SST_concat(df):
    
    incomplete_runs_subj = [] 
    SST_concat = pd.DataFrame()
    incomplete_runs_df = pd.DataFrame()
    df = df[np.logical_and(df.TrialCode != 'BeginFix', \
                           df.TrialCode != 'EndFix')]

    df = df[~df['TrialCode'].isnull()]

    for i in df['NARGUID'].unique(): 
        sub_df = df.loc[df['NARGUID'] == i]
        #add TrialNum col from 1-360
        sub_df['TrialNum'] = np.arange(1, len(sub_df)+1)
        stop_trial_mask = (sub_df['TrialCode'] == 'IncorrectStop') | \
                          (sub_df['TrialCode'] == 'CorrectStop')
        stop_trial_idx = stop_trial_mask[stop_trial_mask == True].index
        sub_df['StopTrial'] = ""
        sub_df['StopTrial'][stop_trial_idx] = np.arange(1, len(sub_df.loc[stop_trial_idx])+ 1)

        if len(~sub_df['TrialNum'].isnull()) == 360: 
            if 'StopTooEarly' not in sub_df['TrialCode'].unique(): 
                SST_concat = SST_concat.append(sub_df)
        else: 
            incomplete_runs_df = incomplete_runs_df.append(sub_df)
            incomplete_runs_subj.append(i)
            
    return SST_concat

In [11]:
for i, v in enumerate([v1, v2, v3, v4, v5, v6]):
    proced = get_SST_concat(v)
    proced.to_csv('merged_data/' + str(i+1) + '_.csv')

/home/sage/anaconda3/envs/home/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/sage/anaconda3/envs/home/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/sage/anaconda3/envs/home/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/s

### After running this for the 6 different versions, analysis continue in Additional Analysis